## Concurrency

In [ ]:
#948

In [ ]:
# We all know that concurrent programming including programming with threads, launching subprocesses and various tricks involving generator functions.




: 

## Starting and Stopping Threads

In [18]:
# The threading library can be used to execute any python callable in its own thread. To do this you create a thread instance and supply the callable that you wish to execute as a target.

# Cod to execute in an independent thread
import time

def countdown(n):
    while n>0:
        print('T minus', n)
        n -= 1
        time.sleep(5)

# Create and launch a thread

from threading import Thread

t = Thread(target=countdown, args=(10,))
t.start()

T minus 10


T minus 9


In [16]:
if t.is_alive():
    print('Still Running')
else:
    print('Completed')

Completed


In [19]:
# For long  running threads or if we want the threads to run forever we should consider making the thread daemonic


t = Thread(target = countdown, args=(10,), daemon=True)
t.start()

T minus 10


T minus 8
T minus 9
T minus 7
T minus 8
T minus 6
T minus 7
T minus 5
T minus 6
T minus 4
T minus 5
T minus 3
T minus 4
T minus 2
T minus 3
T minus 1
T minus 2
T minus 1


## The following [20,21,21] cells describe how to terminate a thread

In [20]:
# If you want to be able to terminate threads, the thread must be programmed to poll for exit at selected points. For example, you might put your thread in a class such  as this:

class CountdownTask:
    def __init__(self):
        self._running = True

    def terminate(self):
        self._running = False
    
    def run(self, n):
        while self._running and n>0:
            print('T-minus',n)
            n -= 1
            time.sleep(3)

c = CountdownTask()
t = Thread(target=c.run, args=(10,), daemon=True)

In [ ]:
t.start()
c.terminate()


Locking with deadlock avoidance

In [4]:
# You are writing a multithreaded program whee threads need to acquire more than one lock at a time while avoiding deadlock


import threading
from contextlib import contextmanager


# Thread local state to stored information on locks already acquired

_local = threading.local()


@contextmanager
def acquire(*locks):
    # Sort locks by object identifier
    locks = sorted(locks, key=lambda x: id(x))

    # Make user the lock order

    acquired = getattr(_local, 'acquired', [])
    if acquired and max(id(lock) for lock in acquired) >= id(locks[0]):
        raise RuntimeError('Lock order violation')


    # Acquire all of the locks
    acquired.extend(locks)
    _local.acquired = acquired

    try:
        for lock in locks:
            lock.acquire()
        yield
    finally:
        # Release locks in reverse order of acquisition

        for lock in reversed(locks):
            lock.release()
        del acquired[-len(locks):]

In [ ]:
# To use this context manager, you simply allocate lock objects in the normal way, but use acquire() function whenever you want to work with one or more locks.


import threading
x_lock = threading.Lock()
y_lock = threading.Lock()

def thread_1():
    while True:
        with acquire(x_lock, y_lock):
            print('Thread-1')

def thread_2():
    while True:
        with acquire(y_lock, x_lock):
            print('Thread-2')


t1 = threading.Thread(target=thread_1)
t1.daemon = True
t1.start()


t2 = threading.Thread(target=thread_2)
t2.daemon = True
t2.start()

In [1]:
# Using a solution, here is a simple deadlock free implementation of the dining philosopher's problem


import threading

# The philosopher thread

def philosopher(left, right):
    while True:
        with acquire(left, right):
            print(threading.currentThread(), 'eating')

# The chopsticks (represented by locks)
NSTICKS  = 5
chopsticks = [threading.Lock() for n in range(NSTICKS)]


# create all of the philosophers

for n in range(NSTICKS):
    t = threading.Thread(target=philosopher, args=(chopsticks[n],chopsticks[(n+1) % NSTICKS]))

Storing Thread Specific State

In [5]:
# You need to store state that's specific to the currently executing thread and not visible to other threads.

from socket import socket, AF_INET, SOCK_STREAM

import threading

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM
        self.local = threading.local()

    def __enter__(self):
        if hasattr(self.local, 'sock'):
            raise RuntimeError('AlreadyConnected')
        self.local.sock = socket(self.family, self.type)
        self.local.sock.connect(self.address)
        return self.local.sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.local.sock.close()
        del self.local.sock